In [59]:
import logging
from luigi import Task
from luigi.parameter import BoolParameter
from luigi.task import ExternalTask
import luigi
from csci_utils.luigi.dask.target import CSVTarget
from csci_utils.luigi.dask.target import ParquetTarget
from csci_utils.luigi.task import Requirement
from csci_utils.luigi.task import Requires
from csci_utils.luigi.task import TargetOutput
from luigi.contrib.s3 import S3Target

In [60]:
import pandas as pd

In [61]:
import dask.dataframe as dd

In [89]:
class ChexpertDataframe(ExternalTask):

    s3_path = 's3://radio-star-csci-e-29/unzipped/'

    output = TargetOutput(
        file_pattern="",
        ext="train.csv",
        target_class=S3Target,
        path=s3_path
    )

In [57]:
os.path.relpath(('C:\\Users\\wmj\\PycharmProjects\\radio-star\\models\\Tasks\\', 'C:\\Users\\wmj\\PycharmProjects\\radio-star\\data\\processed\\')[-1])

'..\\data\\processed'

In [77]:
class ProcessChexpertDf(Task):
    requires = Requires()
    chexpertdf = Requirement(ChexpertDataframe)
    
    output = TargetOutput(
        target_class=ParquetTarget,
        path = "../data/processed/",
        ext="",
        flag=False,
        storage_options=dict(requester_pays=True),
    )
    
    def run(self):
        pathCSV = self.input()["chexpertdf"].path
        ddf = dd.read_csv(pathCSV)
        self.output().write_dask(ddf, compression = "gzip")

In [78]:
class SelectRandomStudy(Task):
    requires = Requires()
    process_chexpert_df = Requirement(ProcessChexpertDf)
    
    output = TargetOutput(
        target_class=luigi.local_target.atomic_file,
        path = "../data/interim/",
)
    
    def run(self):
        pathCSV = self.input()["chexpertdf"].read_parquet()
        ddf = dd.read_csv(pathCSV)
        self.output().write_dask(ddf, compression = "gzip")

In [94]:
df.Path[00]

'CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg'